In [5]:
from qiskit import QuantumCircuit     #to create the circuit
from qiskit_aer import AerSimulator   #to simulate the circuit
import qiskit.quantum_info as qi     #to make the custom unitary
import random as r                   #generates random number for Alice and Bob
import numpy as np                   #for numerical calculation
import sympy as sp                   #for symbolic calulations
from IPython.display import display, Math   #to make certain calculations to display neater
from sympy.physics.quantum import TensorProduct


#conjugate transpose/adjoint of a matrix
def dagger(matrix):
    return matrix.T.conjugate()

#normalizes vectors
def normalize(vector):
    magnitude_squared = 0
    for component in vector:
        magnitude_squared += component * sp.conjugate(component)
    
    magnitude = sp.sqrt(magnitude_squared)
    normalized_vector = vector / magnitude
    return normalized_vector

#Just returns z^2 defined in the paper 
def getz2(n):
    return (sp.exp((sp.I*sp.pi)/(2*n)))**2

#Gets the eigenvectors of an observable
def get_eigenvectors(matrix):
    for eigenvalue, multiplicity, vects in matrix.eigenvects():
        display(Math(f"\\text{{Eigenvalue: }} {sp.latex(eigenvalue)}"))
        for vect in vects:
            display(Math(f"\\text{{Eigenvector: }} {sp.latex(normalize(vect))}"))
    

#returns the phase angle of a complex number from 0 to 2pi.
def order(pair):
    theta = sp.arg(pair[0])  # Compute argument (phase angle) of the eigenvalue
    if theta < 0:
        theta += 2*sp.pi  # Ensure theta is positive
    return theta

#sorts the eigenvectors based on their eigenvalues. the order of the eigen vectors are increasing order of phase angle of the eigenvalues
def get_sorted_normalized_eigenvectors(matrix):
    # Compute eigenvalues and eigenvectors
    eigenvects = matrix.eigenvects()
    
    # Flatten eigenvectors and associate them with their eigenvalues
    eigen_pairs = [(eigenvalue, vect) for eigenvalue, multiplicity, vects in eigenvects for vect in vects]
    
    # Sort eigen_pairs based on the phase angle (theta) of the eigenvalues
    sorted_list = sorted(eigen_pairs, key=order)    
    sorted_eigenvectors = [tup[1] for tup in sorted_list]
    normalized_eigenvectors = []
    
    for vector in sorted_eigenvectors:
        normalized_eigenvectors.append(normalize(vector))
    
    return normalized_eigenvectors
    

#gets unitary based on the order of the eigenvectors input. first eigen vector gets mapped to 0 etc
def get_unitary(ordered_list_of_normalized_eigenvectors):
    dim = len(ordered_list_of_normalized_eigenvectors)
    zero_vector = sp.zeros(dim, 1)
    unitary = sp.zeros(dim, dim)
    for i in range (0,dim):
        zero_vector[i, 0] = 1      #corresponding basis vector
        unitary += zero_vector*dagger(ordered_list_of_normalized_eigenvectors[i])
        zero_vector = sp.zeros(dim, 1)
    
    return unitary 

#gets unitary from any given observable
def get_unitary_from_observable(observable):
    return get_unitary(get_sorted_normalized_eigenvectors(observable))

def generate_state(n):
    gamma = sp.sqrt(2*n + 2/(sp.sin(sp.pi/(2*n))))
    z_n = sp.exp(sp.I*sp.pi/(2*n))
    state = TensorProduct(sp.zeros(n,1),sp.zeros(n,1))
    for i in range (0,n):
        state = state + (1-z_n**(n+2*i+1))*(TensorProduct(cycle_permutation(i,n),cycle_permutation(-i,n)))
    state = (1/gamma)*state
    return state
        
# sends i to i+1 mod n        
def cycle_permutation(i,n):
    zero_vector = sp.zeros(n,1)
    placement = i%n
    zero_vector[placement, 0] = 1
    return zero_vector


def flatten_and_evaluate(nested_list):
    # Recursive function to flatten the nested list
    def flatten(nested_list):
        flat_list = []
        for item in nested_list:
            if isinstance(item, list):
                flat_list.extend(flatten(item))
            else:
                flat_list.append(item)
        return flat_list
    
    # Flatten the nested list
    flat_list = flatten(nested_list)
    numerical_list = []
    
    # Convert SymPy expressions to numerical values
    for number in flat_list:
        numerical_list.append(complex(number.evalf()))
    return numerical_list

def extend_to_C4(state):
    extension = sp.zeros(7,1)
    return state.col_join(extension)

def extend_unitary(unitary):
    extended_row = unitary.row_join(sp.Matrix([0, 0,0]))
    return extended_row.col_join(sp.Matrix([[0, 0, 0,1]]))

def get_omega(n):
    return sp.exp((2*sp.pi*sp.I)/n)

def convert_mod3_to_Z3(player_output):
    if player_output == 0:
        return 1
    elif player_output == 1:
        return sp.exp(sp.I*2*sp.pi/3)
    elif player_output == 2:
        return (sp.exp(sp.I*2*sp.pi/3))**2
    
def check_win(x,y,a,b,n):
    w_n = get_omega(n)
    if x == 0 and y == 0:
        if a == b:
            return 1
    elif x == 0 and y == 1:
        if a * b == 1:
            return 1
    elif x == 1 and y == 0:
        if a == b:
            return 1
    elif x == 1 and y == 1:
        if a * b == w_n:
            return 1
    return

def transform_bits_to_C4(input_dict):
    transformed_dict = {}
    for key, value in input_dict.items():
        bit_pairs = [key[i:i+2] for i in range(0, len(key), 2)]
        C4_representation = ''.join(str(int(pair, 2)) for pair in bit_pairs)
        transformed_dict[C4_representation] = value
    
    return transformed_dict

In [6]:
A0 = sp.Matrix([[0,0,1],[1,0,0],[0,1,0]])
B0 = sp.Matrix([[0,0,1],[1,0,0],[0,1,0]])
A1 = sp.Matrix([[0,0,-getz2(3)],[getz2(3),0,0],[0,getz2(3),0]])
B1 = sp.Matrix([[0,-getz2(3),0],[0,0,getz2(3)],[getz2(3),0,0]])

In [38]:
def CHSH_mod3(rounds,A0,A1,B0,B1):
    A0_U = qi.Operator(extend_unitary(get_unitary_from_observable(A0)).tolist())
    A1_U = qi.Operator(extend_unitary(get_unitary_from_observable(A1)).tolist())
    B0_U = qi.Operator(extend_unitary(get_unitary_from_observable(B0)).tolist())
    B1_U = qi.Operator(extend_unitary(get_unitary_from_observable(B1)).tolist())
    rounds_won=0
    
    for x in range (1,rounds+1):
#Alice and Bob share entangled qutrits from C4
        qc = QuantumCircuit(4,4)
        qc.initialize(optimal_state,[0,1,2,3])
        qc.barrier()
    
#Referee picks 0 or 1 from uniform dist
        alice_input = r.choice([0,1])            
        bob_input = r.choice([0,1])
        print("Round",x)
        print("Alice Input", alice_input)
        print("Bob Input", bob_input)

        if alice_input == 0:
            qc.unitary(A0_U,[0,1], label = 'A0')
        if alice_input == 1:
            qc.unitary(A1_U,[0,1], label = 'A1')
        if bob_input == 0:
            qc.unitary(B0_U, [2,3], label= 'B0')
        if bob_input == 1:
            qc.unitary(B1_U, [2,3], label= 'B1')
 

        qc.measure([0,1,2,3],[0,1,2,3])
        sim = AerSimulator()
        job = sim.run(qc,shots=1)
        results = job.result()
        data = results.get_counts()
        print("Data from circuit:" , data)
        C4_data = (transform_bits_to_C4(data))
        print("Transformed Data", C4_data)


        for bit_string in C4_data:
            bob_output = convert_mod3_to_Z3(int(bit_string[0]))
            alice_output = convert_mod3_to_Z3(int(bit_string[-1]))
            print("Alice Output", alice_output)
            print("Bob Output", bob_output)    
            break
        
        if check_win(alice_input,bob_input,alice_output,bob_output,3) == 1:
            rounds_won = rounds_won + 1
            print("Win")
        
        print("--------------")
        

    print("Win Percentage", (rounds_won/rounds)*100)

In [8]:
optimal_state = [0.474341649025257 - 0.273861278752583j,
 0,
 0,
 0,
 0,
 0,
 0.474341649025257 + 0.273861278752583j,
 0,
 0,
 0.632455532033676,
 0,
 0,
 0,
 0,
 0,
 0]

In [45]:
CHSH_mod3(1000,A0,A1,B0,B1) 

Round 1
Alice Input 0
Bob Input 1
Data from circuit: {'0110': 1}
Transformed Data {'12': 1}
Alice Output exp(-2*I*pi/3)
Bob Output exp(2*I*pi/3)
Win
--------------
Round 2
Alice Input 0
Bob Input 0
Data from circuit: {'0000': 1}
Transformed Data {'00': 1}
Alice Output 1
Bob Output 1
Win
--------------
Round 3
Alice Input 1
Bob Input 1
Data from circuit: {'0001': 1}
Transformed Data {'01': 1}
Alice Output exp(2*I*pi/3)
Bob Output 1
Win
--------------
Round 4
Alice Input 0
Bob Input 1
Data from circuit: {'0110': 1}
Transformed Data {'12': 1}
Alice Output exp(-2*I*pi/3)
Bob Output exp(2*I*pi/3)
Win
--------------
Round 5
Alice Input 1
Bob Input 0
Data from circuit: {'1010': 1}
Transformed Data {'22': 1}
Alice Output exp(-2*I*pi/3)
Bob Output exp(-2*I*pi/3)
Win
--------------
Round 6
Alice Input 0
Bob Input 0
Data from circuit: {'0000': 1}
Transformed Data {'00': 1}
Alice Output 1
Bob Output 1
Win
--------------
Round 7
Alice Input 0
Bob Input 1
Data from circuit: {'0000': 1}
Transformed D

In [44]:
sp.simplify((get_unitary_from_observable(A0)))

Matrix([
[       sqrt(3)/3,        sqrt(3)/3, sqrt(3)/3],
[-sqrt(3)/6 + I/2, -sqrt(3)/6 - I/2, sqrt(3)/3],
[-sqrt(3)/6 - I/2, -sqrt(3)/6 + I/2, sqrt(3)/3]])

In [18]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_ibm_provider import IBMProvider
provider = IBMProvider()
backend = provider.backends(name="ibm_brisbane")[0]

def CHSH_mod3_with_noise(rounds,A0,A1,B0,B1):
    A0_U = qi.Operator(extend_unitary(get_unitary_from_observable(A0)).tolist())
    A1_U = qi.Operator(extend_unitary(get_unitary_from_observable(A1)).tolist())
    B0_U = qi.Operator(extend_unitary(get_unitary_from_observable(B0)).tolist())
    B1_U = qi.Operator(extend_unitary(get_unitary_from_observable(B1)).tolist())
    rounds_won=0
    
    for x in range (1,rounds+1):
        should_continue_outer = False

#Alice and Bob share entangled qutrits from C4
        qc = QuantumCircuit(4,4)
        qc.initialize(optimal_state,[0,1,2,3])
        qc.barrier()
    
#Referee picks 0 or 1 from uniform dist
        alice_input = r.choice([0,1])            
        bob_input = r.choice([0,1])
        print("Round",x)
        print("Alice Input", alice_input)
        print("Bob Input", bob_input)

        if alice_input == 0:
            qc.unitary(A0_U,[0,1], label = 'A0')
        if alice_input == 1:
            qc.unitary(A1_U,[0,1], label = 'A1')
        if bob_input == 0:
            qc.unitary(B0_U, [2,3], label= 'B0')
        if bob_input == 1:
            qc.unitary(B1_U, [2,3], label= 'B1')
 

        qc.measure([0,1,2,3],[0,1,2,3])
         #noisy simulation
        transpiled_circuit = transpile(qc, backend=backend)
        sim = AerSimulator().from_backend(backend,shots=1)
        simulator_result = sim.run(transpiled_circuit).result()
        data = simulator_result.get_counts()
        print("Data from circuit:" , data)
        C4_data = (transform_bits_to_C4(data))
        
        #If Alice or Bob end up having a 3 they lose the round 
        for key, value in C4_data.items():
            # If the key contains a '3', continue to the next iteration
            if '3' in key:
                should_continue_outer = True
                break
        
        if should_continue_outer:
            continue
            
            
        for bit_string in C4_data:
            bob_output = convert_mod3_to_Z3(int(bit_string[0]))
            alice_output = convert_mod3_to_Z3(int(bit_string[-1]))
            print("Alice Output", alice_output)
            print("Bob Output", bob_output)    
            break
        
        if check_win(alice_input,bob_input,alice_output,bob_output,3) == 1:
            rounds_won = rounds_won + 1
            print("Win")
        
        print("--------------")
        

    print("Win Percentage", (rounds_won/rounds)*100)

In [19]:
CHSH_mod3_with_noise(60,A0,A1,B0,B1) 

Round 1
Alice Input 0
Bob Input 0
Data from circuit: {'0000': 1}
Alice Output 1
Bob Output 1
Win
--------------
Round 2
Alice Input 0
Bob Input 1
Data from circuit: {'1010': 1}
Alice Output exp(-2*I*pi/3)
Bob Output exp(-2*I*pi/3)
--------------
Round 3
Alice Input 1
Bob Input 0
Data from circuit: {'0010': 1}
Alice Output exp(-2*I*pi/3)
Bob Output 1
--------------
Round 4
Alice Input 0
Bob Input 0
Data from circuit: {'1010': 1}
Alice Output exp(-2*I*pi/3)
Bob Output exp(-2*I*pi/3)
Win
--------------
Round 5
Alice Input 1
Bob Input 0
Data from circuit: {'0001': 1}
Alice Output exp(2*I*pi/3)
Bob Output 1
--------------
Round 6
Alice Input 1
Bob Input 1
Data from circuit: {'1001': 1}
Alice Output exp(2*I*pi/3)
Bob Output exp(-2*I*pi/3)
--------------
Round 7
Alice Input 0
Bob Input 0
Data from circuit: {'0001': 1}
Alice Output exp(2*I*pi/3)
Bob Output 1
--------------
Round 8
Alice Input 1
Bob Input 0
Data from circuit: {'1011': 1}
Round 9
Alice Input 0
Bob Input 1
Data from circuit: {'00

In [ ]:
        display(transpiled_circuit.draw())
